In [1]:
from datasets import load_dataset,Dataset
from datasets.features import Features,Value
import os

In [2]:
data_path = os.getenv("NEWS_DATA_PATH")

In [3]:
columns = ["idx","article_idx","date","year","month","day","author","title","article","url","section","publication"]
features_dict = Features({i: Value("int16")  if i in ["idx","article_idx"] else Value("string") for i in columns})

In [4]:
df = load_dataset("csv",data_files=[data_path],skiprows=[0,2_324_812],column_names=columns,features=features_dict)

Using custom data configuration default-743872de51535b2e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/rjac/.cache/huggingface/datasets/csv/default-743872de51535b2e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
df.push_to_hub("rjac/all-the-news-2-1-Component-one",max_shard_size="250MB",private=False,token=os.getenv("AUTH_TOKEN"))

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/36 [00:00<?, ?it/s]